# ติดตั้งและ Import Library

In [ ]:
!pip install pythainlp -q
!pip install lime -q
!pip install imbalanced-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import re
import glob
import csv
import string
import joblib
import warnings
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import lime
import lime.lime_text

from google.colab import drive

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib
import os

# เชื่อมต่อ Drive & โหลด Repository
Repository จาก https://github.com/ssivakorn/Thai-SMS-spam-filter.git

In [ ]:
try:
    drive.mount('/content/drive')

    # ที่จัดเก็บหลักของโปรเจกต์
    DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SMS_Scam_Final_Project/'
    os.makedirs(DRIVE_PROJECT_PATH, exist_ok=True)

    print(f"เชื่อมต่อสำเร็จ - ไฟล์จะถูกจัดเก็บไว้ที่ : {DRIVE_PROJECT_PATH}")

except Exception as e:
    print(f"\nเกิดข้อผิดพลาดในการเชื่อมต่อ : {e}")

Mounted at /content/drive
เชื่อมต่อสำเร็จ - ไฟล์จะถูกจัดเก็บไว้ที่ : /content/drive/MyDrive/SMS_Scam_Final_Project/


In [ ]:
print("รวบรวมข้อมูล")

repo_name = 'Thai-SMS-spam-filter'
repo_path = os.path.join(DRIVE_PROJECT_PATH, repo_name)
repo_url = 'https://github.com/ssivakorn/Thai-SMS-spam-filter.git'

if not os.path.exists(repo_path):
    print(f"ไม่พบข้อมูลกำลังโหลด {repo_path}")
    !git clone {repo_url} {repo_path} -q
    print("โหลด Repository สำเร็จ")
else:
    print(f"พบข้อมูลอยู่แล้ว")

# อ่านข้อมูลจากทุกไฟล์
data_path = os.path.join(repo_path, 'data/raw_data/')
all_files = glob.glob(os.path.join(data_path, '*.txt'))
all_data = []
print(f"\nพบไฟล์ข้อมูลทั้งหมด {len(all_files)} ไฟล์ กำลังโหลด")

for file_path in all_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t', 1)
            if len(parts) < 2: parts = line.strip().split('|', 1)
            if len(parts) == 2:
                all_data.append({'label': parts[0], 'message': parts[1]})

# สร้าง DataFrame
if all_data:
    print(all_data)
    df = pd.DataFrame(all_data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    print("\nโหลดและรวมข้อมูลสำเร็จ")
    print(f"ข้อมูลทั้งหมด {len(df)} แถว")
    display(df['label'].value_counts())
else:
    print("\nไม่สามารถโหลดข้อมูลได้")


รวบรวมข้อมูล
พบข้อมูลอยู่แล้ว

พบไฟล์ข้อมูลทั้งหมด 17 ไฟล์ กำลังโหลด
[{'label': 'spam', 'message': 'ดูไพ่ยิปซีกับ อ.อุมาวดี ศิริอักษร แบบแม่นเว่อร์! *155*34# 9บ/sms/วัน'}, {'label': 'spam', 'message': 'ดูไพ่ยิปซีกับ อ.อุมาวดี ศิริอักษร แบบแม่นเว่อร์! *155*34# 9บ/sms/วัน'}, {'label': 'spam', 'message': 'ดูดวงรายวันกับโหรชื่อดังแถมได้ลุ้นทอง1สลึง สมัคร*155*34# 9บ/sms/วัน'}, {'label': 'scam', 'message': 'b72kk.com เว็บตรงแจกฟรี 9บาท รับเพิ่ม500% คลิก https://lin.ee/lmMFU5k'}, {'label': 'scam', 'message': 'b76kk.com ฟรี177777 PG JILI SEXY คลิ๊กแจ้งรัU https://lin.ee/qJHNErv'}, {'label': 'scam', 'message': 'JOOXVIP ฟรี7วัน ฟรีเน็ต จากนั้นแค่วันละ5บาท คลิก m.ais.co.th/4RHroRah8'}, {'label': 'OK', 'message': 'เฉพาะคุณ เพียงมียอดใช้งาน 10 บาท ภายใน 18/03/2565 รับโบนัสเงินคืน \ufeff10 บาท โบนัสใช้โทร และส่ง SMS ในประเทศ ได้ 2 วัน'}, {'label': 'OK', 'message': 'รับวันใช้งานเพิ่ม 30 วัน เมื่อสะสมยอดใช้งานครบ 10 บาท ถึง 27/03/25\ufeff65 ตรวจสอบยอดเงินและวันคงเหลือกด*121# ฟรี'}, {'label': 'OK', 'me

,count
label,
OTP,1880
scam,1431
OK,1024
spam,608


# ตัด Emoji & สร้าง Features & กำหนด Label และบันทึก label_encoder

In [ ]:
def text_process(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # ตัด Emoji
    emoji_pattern = re.compile(
        "["
        u"\\U0001F600-\\U0001F64F"
        u"\\U0001F300-\\U0001F5FF"
        u"\\U0001F680-\\U0001F6FF"
        u"\\U0001F1E0-\\U0001F1FF"
        u"\\U00002702-\\U000027B0"
        u"\\U000024C2-\\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # แทนที่ emoji ด้วยข้อความว่าง

    tokens = word_tokenize(text, engine='newmm')
    stopwords_list = list(thai_stopwords())
    punctuations_to_remove = string.punctuation.replace('%', '')

    processed_tokens = [
        word for word in tokens
        if word not in stopwords_list and word not in punctuations_to_remove
    ]

    return " ".join(processed_tokens)


def create_features(df_input: pd.DataFrame) -> pd.DataFrame:
    df_out = df_input.copy()  # ทำสำเนาของ DataFrame

    # Statistical Features
    df_out['message_length'] = df_out['message'].str.len()
    # นับจำนวนตัวอักษรทั้งหมดในข้อความ
    # Spam/Scam มักจะมีความยาวที่ผิดปกติ อาจจะสั้นหรือยาวเกินไปเพื่อกระตุ้นความสนใจ

    df_out['digit_count'] = df_out['message'].str.count(r'\\d')
    # นับจำนวนตัวเลขทั้งหมดในข้อความ
    # Spam/Scam มักจะมีตัวเลขเยอะ เช่น เบอร์โทรศัพท์, ราคา, รหัสโปรโมชั่น, หรือจำนวนเงินกู้

    df_out['uppercase_ratio'] = df_out['message'].str.count(r'[A-Z]') / df_out['message_length']
    # คำนวณสัดส่วนตัวอักษรภาษาอังกฤษพิมพ์ใหญ่
    # Spam/Scam มักใช้ตัวพิมพ์ใหญ่จำนวนมาก (เช่น FREE!, ด่วน!) เพื่อดึงดูดความสนใจและสร้างความรู้สึกเร่งด่วน

    df_out['repeated_punctuation_count'] = df_out['message'].str.count(r'([!?.])\\1{1,}')
    # นับจำนวนการใช้อักขระพิเศษซ้ำๆ (เช่น !!, ???)
    # เป็นลักษณะที่พบได้บ่อยในข้อความ Spam/Scam เพื่อเน้นย้ำและกระตุ้นอารมณ์ แต่ไม่ค่อยพบในการสื่อสารทั่วไป

    # Pattern-based Features
    df_out['url_count'] = df_out['message'].str.count(r'https?://\\S+|www\\.\\S+|\\S+\\.\\S+')
    # นับจำนวนลิงก์ URL ในข้อความ
    # เป็นช่องทางหลักในการหลอกให้ผู้ใช้คลิกเข้าไปยังเว็บไซต์ปลอมหรือเว็บโฆษณา

    short_url_pattern = 'bit.ly|cutt.ly|t.co|rebrand.ly|shorturl.asia|line.me|lin.ee'
    df_out['has_short_url'] = df_out['message'].str.contains(short_url_pattern, case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีลิงก์ย่อหรือไม่
    # มิจฉาชีพนิยมใช้ลิงก์ย่อเพื่อปกปิด URL ปลายทางที่แท้จริง ซึ่งเป็นพฤติกรรมที่น่าสงสัย

    phone_pattern = r'(\\d{2,3}-\\d{3,4}-\\d{4}|\\d{9,10})'
    df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
    # ตรวจสอบว่ามีเบอร์โทรศัพท์หรือไม่
    # Spam/Scam มักจะใส่เบอร์โทรเพื่อกระตุ้นให้โทรกลับไปสอบถามข้อมูลเพิ่มเติม

    # Keyword-based Features
    df_out['has_line_keyword'] = df_out['message'].str.contains('Line|ไลน์|แอดไลน์', case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีคำว่า "Line" หรือ "ไลน์" หรือไม่
    # การชักชวนให้แอดไลน์เป็นช่องทางยอดนิยมของมิจฉาชีพในประเทศไทย

    advertisement_keywords = ['โปรโมชั่น', 'ส่วนลด', 'ช้อป', 'สาขา', 'คลิกเลย', 'แบรนด์', 'คอลเลคชั่น', 'โค้ด']
    df_out['advertisement_words_count'] = df_out['message'].str.count('|'.join(advertisement_keywords)).astype(int)
    # นับจำนวนคำโฆษณาในข้อความ
    # Spam มักเป็นโฆษณาที่ไม่พึงประสงค์ การมีคำเหล่านี้เพิ่มโอกาสการเป็น Spam

    brand_keywords = ['Lazada', 'Shopee', 'AIS', 'dtac', 'True', 'KFC', 'Grab', 'Central', 'Uniqlo', 'SCB', 'KBank']
    df_out['is_known_brand'] = df_out['message'].str.contains('|'.join(brand_keywords), case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีชื่อแบรนด์ที่รู้จักหรือไม่
    # มิจฉาชีพมักแอบอ้างชื่อแบรนด์ดังเพื่อสร้างความน่าเชื่อถือในการหลอกลวง (Phishing)

    df_out['is_gambling_keyword'] = df_out['message'].str.contains('เว็บตรง|PG|JOKER|สล็อต|บาคาร่า|คาสิโน|แทงหวย|ฝากถอน|ยูสใหม่|แตกง่าย', case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีคำเกี่ยวกับการพนันหรือไม่
    # เป็นตัวบ่งชี้ที่ชัดเจนของข้อความ Scam ที่ชักชวนให้เล่นการพนันออนไลน์

    df_out['is_loan_keyword'] = df_out['message'].str.contains('สินเชื่อ|เงินด่วน|อนุมัติไว|วงเงินกู้|กู้ได้|ไม่เช็คบูโร', case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีคำเกี่ยวกับสินเชื่อหรือไม่
    # เป็นลักษณะเด่นของข้อความ Scam ที่หลอกลวงเรื่องเงินกู้นอกระบบ

    df_out['is_job_offer_keyword'] = df_out['message'].str.contains('ทำงานที่บ้าน|รายได้ต่อวัน|part-time|พาร์ทไทม์|รับสมัคร', case=False, regex=True).astype(int)
    # ตรวจสอบว่ามีคำเกี่ยวกับงานพาร์ทไทม์หรือไม่
    # เป็นกลวิธีที่พบบ่อยในข้อความ Scam ที่หลอกให้สมัครงานที่ไม่มีอยู่จริง

    emotional_words = ['ฟรี', 'รางวัล', 'แจก', 'โบนัส', 'เครดิต', 'รับสิทธิ์', 'เงินคืน', 'ด่วน', 'ภายใน', 'จำกัด', 'เท่านั้น', 'ผิดปกติ', 'ถูกระงับ', 'มีปัญหา']
    df_out['emotional_words_count'] = df_out['message'].str.count('|'.join(emotional_words)).astype(int)
    # นับจำนวนคำกระตุ้นอารมณ์ในข้อความ
    # Spam/Scam มักใช้คำเหล่านี้เพื่อสร้างความรู้สึกเร่งรีบ, โลภ, หรือกลัว ทำให้ผู้รับขาดความระมัดระวัง

    obfuscated_pattern = r'\\b[ก-ฮ]*[a-zA-Z0-9]+[ก-ฮ]+[a-zA-Z0-9]*\\b'
    df_out['obfuscated_word_count'] = df_out['message'].str.count(obfuscated_pattern)
    # นับจำนวนคำที่ใช้ตัวอักษรผสมกันผิดปกติ (เช่น ฟsี, โuนัส)
    # เป็นเทคนิคที่มิจฉาชีพใช้บ่อยเพื่อหลบเลี่ยงระบบตรวจจับคำ (Keyword Filtering) ซึ่งเป็นสัญญาณของ Scam ที่ชัดเจน

    # ประมวลผลข้อความ (ต้องทำก่อนคำนวณ Word Diversity)
    df_out['processed_message'] = df_out['message'].apply(text_process)

    # คำนวณความหลากหลายของคำศัพท์
    df_out['word_diversity_ratio'] = df_out['processed_message'].apply(
        lambda x: len(set(x.split())) / (len(x.split()) + 1e-6) # +1e-6 เพื่อป้องกันการหารด้วยศูนย์
    )
    # คำนวณสัดส่วนคำที่ไม่ซ้ำกันต่อจำนวนคำทั้งหมด
    # ข้อความ Spam มักใช้คำซ้ำๆ (เช่น ฟรี ฟรี ฟรี) ทำให้มีความหลากหลายของคำศัพท์ต่ำ

    return df_out


print("\nกำลังสร้าง Features")

if 'df' in locals():
    df_featured = create_features(df)
    print("\nสร้าง Features สำเร็จ")
else:
    print("\nไม่พบ DataFrame")



กำลังสร้าง Features


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)



สร้าง Features สำเร็จ


In [ ]:
print("กำหนด Labels (Label Definition)")

if 'df_featured' in locals():
    label_encoder = LabelEncoder()
    df_featured['label_encoded'] = label_encoder.fit_transform(df_featured['label'])
    print("แปลง Label เรียบร้อย")

    encoder_path = os.path.join(DRIVE_PROJECT_PATH, 'label_encoder.joblib')
    joblib.dump(label_encoder, encoder_path)  # บันทึกตัว label_encoder เป็นไฟล์ .joblib เพื่อเก็บไว้ใช้งาน
    print(f"\nบันทึก Label Encoder ลง Drive สำเร็จ")
else:
    print("ไม่พบ DataFrame")


กำหนด Labels (Label Definition)
แปลง Label เรียบร้อย

บันทึก Label Encoder ลง Drive สำเร็จ


# ทดลองหาสัดส่วนและโมเดลที่ดีที่สุด

In [ ]:
print("ตั้งค่าและทดลองที่สัดส่วน 60:40")

# ตรวจสอบว่า DataFrame
if 'df_featured' in locals():
    # การตั้งค่า
    # กำหนดข้อมูล Input (X) และ Output (y)
    X = df_featured['processed_message']
    y = df_featured['label_encoded']

    # กำหนดโมเดลที่จะใช้
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "Neural Network": MLPClassifier(random_state=42, max_iter=500),
        "Naive Bayes": MultinomialNB()
    }

    # สร้าง Dictionary เก็บผลลัพธ์
    experiment_results = {}

    # ทดลองที่สัดส่วน 60:40
    print("\nทดลองที่สัดส่วน 60:40 ")
    ratio_key = "60:40"
    experiment_results[ratio_key] = {}

    # แบ่งข้อมูลและแปลงเป็น TF-IDF
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42, stratify=y)
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    # ทดสอบทุกโมเดล
    for model_name, model in models.items():
        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        accuracy = accuracy_score(y_test, y_pred)
        experiment_results[ratio_key][model_name] = accuracy
        print(f"  - โมเดล : {model_name:<15} | Accuracy : {accuracy:.5f}")

else:
    print("ไม่พบ DataFrame")


ตั้งค่าและทดลองที่สัดส่วน 60:40

ทดลองที่สัดส่วน 60:40 


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  - โมเดล : Logistic Regression | Accuracy : 0.91153
  - โมเดล : Random Forest   | Accuracy : 0.90344
  - โมเดล : Neural Network  | Accuracy : 0.91860
  - โมเดล : Naive Bayes     | Accuracy : 0.86653


In [ ]:
print("\nทดลองที่สัดส่วน 70:30")

# กำหนดคีย์สำหรับเก็บผลลัพธ์
ratio_key = "70:30"
experiment_results[ratio_key] = {}

# แบ่งข้อมูลและแปลงเป็น TF-IDF
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# ทดสอบโมเดล
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    experiment_results[ratio_key][model_name] = accuracy
    print(f"  - โมเดล : {model_name:<15} | Accuracy: {accuracy:.5f}")



ทดลองที่สัดส่วน 70:30


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  - โมเดล : Logistic Regression | Accuracy: 0.92245
  - โมเดล : Random Forest   | Accuracy: 0.91504
  - โมเดล : Neural Network  | Accuracy: 0.92785
  - โมเดล : Naive Bayes     | Accuracy: 0.87525


In [ ]:
print("\nทดลองที่สัดส่วน 80:20")

# กำหนดคีย์สำหรับเก็บผลลัพธ์
ratio_key = "80:20"
experiment_results[ratio_key] = {}

# แบ่งข้อมูลและแปลงเป็น TF-IDF
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# ทดสอบโมเดล
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    experiment_results[ratio_key][model_name] = accuracy
    print(f"  - โมเดล : {model_name:<15} | Accuracy: {accuracy:.5f}")



ทดลองที่สัดส่วน 80:20


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  - โมเดล : Logistic Regression | Accuracy: 0.92214
  - โมเดล : Random Forest   | Accuracy: 0.91709
  - โมเดล : Neural Network  | Accuracy: 0.92720
  - โมเดล : Naive Bayes     | Accuracy: 0.88574


## สรุป
สัดส่วนและโมเดลที่ ค่า Accuracy เยอะที่สุดคือ สัดส่วน 70:30 โมเดล Neural Network ค่า Accuracy อยู่ที่ 0.92785

# เตรียมข้อมูลสำหรับ Train และ Test

In [ ]:
if 'df_featured' in locals():
    X = df_featured['processed_message']
    y = df_featured['label_encoded']

    # เลือก test_size เป็น 0.3 ตามการทดลองสัดส่วน
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y)
    print(f"แบ่งข้อมูลเป็น Train: {len(X_train)} แถว, Test: {len(X_test)} แถว (สัดส่วน 70:30)")

    # แปลงข้อความเป็นตัวเลขด้วย TF-IDF
    tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    print("\nแปลงข้อความเป็น TF-IDF เรียบร้อย")
else:
    print("ไม่พบ DataFrame 'df_featured'")

แบ่งข้อมูลเป็น Train: 3460 แถว, Test: 1483 แถว (สัดส่วน 70:30)


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



แปลงข้อความเป็น TF-IDF เรียบร้อย


# สร้างโมเดล
สร้างโมเดล Logistic Regression,Random Forest,Neural Network,Naive Bayes
เพื่อตรวจสอบ Classification Report แต่ละตัว

In [ ]:
models = {
   "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Neural Network": MLPClassifier(random_state=42, max_iter=500),
    "Naive Bayes": MultinomialNB()
}

results = {}  # สร้าง dictionary ว่าง

# สอนและประเมินผลทุกโมเดล
for model_name, model in models.items():
    print(f"\nกำลังสอนโมเดล : {model_name}")
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    results[model_name] = accuracy

    # แสดงรายงาน
    print(f"Accuracy : {accuracy:.4f}")
    print("\nClassification Report :")
    # ใช้ target_names=label_encoder.classes_ เพื่อแสดงชื่อ Label แทนตัวเลข
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

#สรุปผล
best_model_name = max(results, key=results.get)
print(f"โมเดลที่ดีที่สุดจากการทดลองคือ : -- {best_model_name} --")
print(f"   ด้วย Accuracy รวม = {results[best_model_name]:.4f}")



กำลังสอนโมเดล : Logistic Regression
Accuracy : 0.9225

Classification Report :
              precision    recall  f1-score   support

          OK       0.89      0.88      0.88       307
         OTP       0.98      0.99      0.99       564
        scam       0.90      0.94      0.92       429
        spam       0.83      0.73      0.78       183

    accuracy                           0.92      1483
   macro avg       0.90      0.89      0.89      1483
weighted avg       0.92      0.92      0.92      1483


กำลังสอนโมเดล : Random Forest
Accuracy : 0.9150

Classification Report :
              precision    recall  f1-score   support

          OK       0.90      0.85      0.87       307
         OTP       0.98      0.99      0.99       564
        scam       0.87      0.94      0.90       429
        spam       0.85      0.72      0.78       183

    accuracy                           0.92      1483
   macro avg       0.90      0.88      0.89      1483
weighted avg       0.91      0.

# การปรับจูนไฮเปอร์พารามิเตอร์ (Hyperparameter Tuning)

In [ ]:
# ปรับจูน Hyperparameter สำหรับ Neural Network
# กำหนดพารามิเตอร์ที่ต้องการค้นหา (Parameter Grid)
param_grid_nn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 25)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

# สร้าง GridSearchCV เพื่อค้นหาพารามิเตอร์ที่ดีที่สุด
grid_search = GridSearchCV(
    MLPClassifier(random_state=42, max_iter=500),  # สร้างโมเดล Neural Network พร้อมกำหนด max_iter
    param_grid_nn,
    cv=3,
    verbose=2,
    n_jobs=-1
)

# ค้นหาและฝึกโมเดลด้วยข้อมูล TF-IDF ที่แปลงแล้ว
grid_search.fit(X_train_tfidf, y_train)

# ดึงโมเดลที่ดีที่สุดจากผลการค้นหา
best_tuned_model = grid_search.best_estimator_

print(f"\nHyperparameters ที่ดีที่สุด: {grid_search.best_params_}")
print("โมเดลที่ดีที่สุดถูกเก็บในตัวแปร 'best_tuned_model'")


Fitting 3 folds for each of 24 candidates, totalling 72 fits

Hyperparameters ที่ดีที่สุด: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}
โมเดลที่ดีที่สุดถูกเก็บในตัวแปร 'best_tuned_model'


In [ ]:
# ประเมินประสิทธิภาพโมเดลที่ปรับจูนแล้ว
# ประเมินความเสถียรด้วย 5-Fold Cross-Validation บนข้อมูล Train
cv_scores = cross_val_score(best_tuned_model, X_train_tfidf, y_train, cv=5)
print(f"\nAccuracy เฉลี่ย (Cross-Validation) : {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.2f})")

# ประเมินประสิทธิภาพสุดท้ายบน Test Set (ข้อมูลที่ไม่เคยเห็น)
y_pred_final = best_tuned_model.predict(X_test_tfidf)

# แสดง Classification Report
print("\nClassification Report บน Test Set:\n")
print(classification_report(y_test, y_pred_final, target_names=label_encoder.classes_))


Accuracy เฉลี่ย (Cross-Validation) : 0.9118 (+/- 0.01)

Classification Report บน Test Set:

              precision    recall  f1-score   support

          OK       0.87      0.91      0.89       307
         OTP       0.99      0.99      0.99       564
        scam       0.94      0.92      0.93       429
        spam       0.82      0.80      0.81       183

    accuracy                           0.93      1483
   macro avg       0.90      0.90      0.90      1483
weighted avg       0.93      0.93      0.93      1483



# สร้าง Pipeline โหลดโมเดล และทดสอบ

In [ ]:
print("เริ่มต้นสร้าง Pipeline...")

# เตรียมข้อมูล Input (X) และ Output (y)
# เลือกคอลัมน์ทั้งหมดที่จะใช้เป็นฟีเจอร์ ไม่รวม label และข้อความดิบ
feature_columns = [
    'processed_message', 'message_length', 'digit_count', 'uppercase_ratio',
    'repeated_punctuation_count', 'url_count', 'has_short_url', 'has_phone',
    'has_line_keyword', 'advertisement_words_count', 'is_known_brand',
    'is_gambling_keyword', 'is_loan_keyword', 'is_job_offer_keyword',
    'emotional_words_count', 'obfuscated_word_count', 'word_diversity_ratio'
]
X = df_featured[feature_columns]
y = df_featured['label_encoded']
print(f"ใช้ฟีเจอร์ทั้งหมด {len(feature_columns)} ตัวในการสร้างโมเดล")

# กำหนดประเภทของฟีเจอร์
text_feature = 'processed_message'
numeric_features = [col for col in X.columns if col != text_feature]
print(f"ฟีเจอร์ประเภทข้อความ : '{text_feature}'")
print(f"ฟีเจอร์ประเภทตัวเลข : {len(numeric_features)} ตัว")

# สร้าง Preprocessor ด้วย ColumnTransformer
# ขั้นตอนนี้จะรวมการประมวลผลฟีเจอร์ข้อความและตัวเลขเข้าด้วยกัน
preprocessor = ColumnTransformer(
    transformers=[
        # Pipeline ย่อยสำหรับประมวลผลข้อความ
        ('tfidf', TfidfVectorizer(tokenizer=word_tokenize), text_feature),
        # Pipeline ย่อยสำหรับประมวลผลตัวเลข
        ('numeric', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'
)
print("สร้าง Preprocessor สำหรับรวมฟีเจอร์สำเร็จ")

# สร้าง Pipeline ที่สมบูรณ์ที่สุด
# ใช้ ImbPipeline เพื่อจัดการ SMOTE และใช้ best_tuned_model
final_production_pipeline = ImbPipeline([
    ('preprocessor', preprocessor),          # ประมวลผลและรวมฟีเจอร์ทั้งหมด
    ('smote', SMOTE(random_state=42)),       # สร้างข้อมูลสังเคราะห์เพื่อแก้ปัญหา Imbalance
    ('clf', best_tuned_model)                # ทำนายผลด้วย Neural Network ที่ปรับจูนแล้ว
])
print("สร้าง Pipeline ที่รวมทุกขั้นตอนสมบูรณ์")

# สอนโมเดลด้วยข้อมูลทั้งหมด
print("กำลังสอนโมเดลด้วยข้อมูลทั้งหมด...")
final_production_pipeline.fit(X, y)
print("สอนโมเดลสำเร็จ")

# บันทึก Pipeline
# บันทึกเป็นไฟล์ใหม่เพื่อไม่ให้ทับของเดิม
pipeline_path_final = os.path.join(DRIVE_PROJECT_PATH, 'final_sms_classifier_production.joblib')
joblib.dump(final_production_pipeline, pipeline_path_final)
print(f"บันทึก Pipeline เวอร์ชัน Production ลงที่ '{pipeline_path_final}' เรียบร้อยแล้ว")

เริ่มต้นสร้าง Pipeline...
ใช้ฟีเจอร์ทั้งหมด 17 ตัวในการสร้างโมเดล
ฟีเจอร์ประเภทข้อความ : 'processed_message'
ฟีเจอร์ประเภทตัวเลข : 16 ตัว
สร้าง Preprocessor สำหรับรวมฟีเจอร์สำเร็จ
สร้าง Pipeline ที่รวมทุกขั้นตอนสมบูรณ์
กำลังสอนโมเดลด้วยข้อมูลทั้งหมด...


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


สอนโมเดลสำเร็จ
บันทึก Pipeline เวอร์ชัน Production ลงที่ '/content/drive/MyDrive/SMS_Scam_Final_Project/final_sms_classifier_production.joblib' เรียบร้อยแล้ว


In [ ]:
def analyze_and_log_sms(sms_text, pipeline_path, encoder_path, log_file_path):
    # โหลดโมเดลและ Encoder
    pipeline = joblib.load(pipeline_path)
    le = joblib.load(encoder_path)

    # สร้าง DataFrame จากข้อความที่รับเข้ามา
    input_df = pd.DataFrame([{'message': sms_text}])

    # เรียกใช้ฟังก์ชัน create_features เพื่อสร้างฟีเจอร์
    features_df = create_features(input_df)

    # ทำนายผลโดยใช้ DataFrame ที่มีฟีเจอร์
    prediction_encoded = pipeline.predict(features_df)[0]
    prediction = le.inverse_transform([prediction_encoded])[0]

    # สร้างฟังก์ชัน predict_proba สำหรับ LIME ให้เรียกใช้ create_features ด้วย
    def lime_predict_fn(texts):
        lime_df = create_features(pd.DataFrame(texts, columns=['message']))
        return pipeline.predict_proba(lime_df)

    # อธิบายด้วย LIME
    reason_string = "N/A"
    try:
        explainer = lime.lime_text.LimeTextExplainer(class_names=le.classes_, bow=False, split_expression=word_tokenize)
        exp = explainer.explain_instance(
            sms_text,
            lime_predict_fn,
            num_features=5,
            labels=[prediction_encoded]
        )
        reason_parts = [f"'{word} : {weight}'" for word, weight in exp.as_list(label=prediction_encoded) if weight > 0]
        reason_string = ", ".join(reason_parts) if reason_parts else "ไม่พบคำสำคัญที่เป็นเหตุผล"
    except Exception as e:
        reason_string = f"เกิดข้อผิดพลาดใน LIME : {e}"

    # บันทึก Log
    try:
        file_exists = os.path.exists(log_file_path)
        with open(log_file_path, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(["timestamp", "prediction", "message", "reason"])
            writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), prediction, sms_text, reason_string])
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการบันทึก Log : {e}")

    return prediction, reason_string

print("สร้างฟังก์ชันเวอร์ชันที่ถูกต้องสำเร็จ")

สร้างฟังก์ชันเวอร์ชันที่ถูกต้องสำเร็จ


In [ ]:
# กำหนดที่อยู่ของไฟล์ทั้งหมดในเซลล์นี้
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SMS_Scam_Final_Project/'
pipeline_path = os.path.join(DRIVE_PROJECT_PATH, 'final_sms_classifier_production.joblib')
encoder_path = os.path.join(DRIVE_PROJECT_PATH, 'label_encoder.joblib')
LOG_FILE = os.path.join(DRIVE_PROJECT_PATH, 'prediction_log.csv')

print("ทดสอบระบบ")

sms_to_test = "โทรด่วน *191# เพียง 3บ/นาที! "

# เรียกใช้ฟังก์ชันเวอร์ชันใหม่ โดยส่ง path ทั้งหมดเข้าไปด้วย
pred, reason = analyze_and_log_sms(sms_to_test, pipeline_path, encoder_path, LOG_FILE)

print(f"ข้อความ : '{sms_to_test}'")
print(f" -> ผลการทำนาย : {pred}")
print(f" -> เหตุผลหลัก : {reason}")
print(f"\nบันทึกผลลง '{LOG_FILE}' เรียบร้อย")

ทดสอบระบบ


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)


ข้อความ : 'โทรด่วน *191# เพียง 3บ/นาที! '
 -> ผลการทำนาย : spam
 -> เหตุผลหลัก : 'โทร : 0.0864551770826618', 'บ : 0.0821626577661985', 'ด่วน : 0.06896393287986533', '*191# : 0.06182999118838255', '! : 0.05186074663102322'

บันทึกผลลง '/content/drive/MyDrive/SMS_Scam_Final_Project/prediction_log.csv' เรียบร้อย


# ทดสอบจากข้อมูล

In [ ]:
test_dataset = [
    # ประเภท : OK
    {'message': 'พัสดุของคุณ TH123456789 กำลังจะจัดส่งถึงคุณภายในวันนี้', 'label': 'OK'},
    {'message': 'คำสั่งซื้อ #98765 ของคุณจัดส่งสำเร็จแล้ว ขอบคุณที่ใช้บริการ', 'label': 'OK'},
    {'message': 'เรียนคุณสมชาย ขณะนี้ธนาคารกำลังปิดปรับปรุงระบบ ขออภัยในความไม่สะดวก', 'label': 'OK'},
    {'message': 'ค่าน้ำประปาของคุณเดือนนี้คือ 350 บาท กำหนดชำระภายในวันที่ 15', 'label': 'OK'},
    {'message': 'ผลตรวจสุขภาพของคุณออกแล้ว สามารถดูได้ที่แอปโรงพยาบาล', 'label': 'OK'},

    # ประเภท : OTP
    {'message': 'รหัส OTP ของคุณคือ 882154 สำหรับการทำรายการ', 'label': 'OTP'},
    {'message': 'Lazada: 123456 คือรหัสยืนยันของคุณ (ref: ABC)', 'label': 'OTP'},
    {'message': '[ธนาคารกรุงไทย] รหัส OTP ของคุณคือ 604831 สำหรับทำรายการ Krungthai NEXT', 'label': 'OTP'},
    {'message': '(SCB) OTP: 901-222 ใช้สำหรับสมัครบริการ', 'label': 'OTP'},
    {'message': 'Google: G-587123 คือรหัสยืนยันของคุณ', 'label': 'OTP'},

    # ประเภท : scam
    {'message': 'บัญชีธนาคารของคุณถูกระงับ กรุณาคลิก line.ee/xyz เพื่อยืนยันตัวตน', 'label': 'scam'},
    {'message': 'คุณได้รับสิทธิ์สินเชื่อ 50,000 บาท อนุมัติใน 5 นาที ไม่เช็คบูโร แอดไลน์ @loanfast', 'label': 'scam'},
    {'message': 'หารายได้เสริมง่ายๆ วันละ 800-1000 บาท แค่กดดูยูทูป สนใจแอดไลน์', 'label': 'scam'},
    {'message': 'คุณมีพัสดุจาก DHL ตกค้างเนื่องจากที่อยู่ไม่ชัดเจน โปรดยืนยันที่ cutt.ly/update', 'label': 'scam'},
    {'message': 'บัญชีของคุณมีความผิดปกติ กรุณายืนยันตัวตนด่วน >> www.bkk-safe-verify.com', 'label': 'scam'},

    # ประเภท : spam
    {'message': 'FLASH SALE! เสื้อผ้าแบรนด์ลด 70% วันเดียวเท่านั้น ช้อปเลย', 'label': 'spam'},
    {'message': 'ช้อป Lazada Shopee ลดสูงสุด 90% ทักเลย LINE: @shop-now', 'label': 'spam'},
    {'message': 'โปรสุดคุ้ม! เน็ตไม่อั้น โทรฟรีเพียงวันละ 9 บาท สมัครกด *123#', 'label': 'spam'},
    {'message': 'โหลดแอปดูดวงฟรี แม่นขั้นเทพ พร้อมคำทำนายเฉพาะคุณ >> www.horolive.today', 'label': 'spam'},
    {'message': 'ดูหนังฟรีไม่อั้นเดือนแรก สมัครเลยวันนี้', 'label': 'spam'},
]

print(f"สร้างข้อมูลทดสอบเสร็จสิ้น จำนวน {len(test_dataset)} ข้อความ")

สร้างข้อมูลทดสอบเสร็จสิ้น จำนวน 20 ข้อความ


In [ ]:
# กำหนดที่อยู่ของไฟล์ทั้งหมด
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SMS_Scam_Final_Project/'
pipeline_path = os.path.join(DRIVE_PROJECT_PATH, 'final_sms_classifier_production.joblib')
encoder_path = os.path.join(DRIVE_PROJECT_PATH, 'label_encoder.joblib')
LOG_FILE = os.path.join(DRIVE_PROJECT_PATH, 'prediction_log.csv')

# ตรวจสอบชุดข้อมูลทดสอบ
if 'test_dataset' in locals():
    true_labels = []
    predicted_labels = []

    print("\nเริ่มการทดสอบระบบแบบกลุ่ม")

    # วนลูปทดสอบทีละข้อความ
    for item in test_dataset:
        sms_text = item['message']
        true_label = item['label']

        # สร้าง DataFrame ชั่วคราวเพื่อดึงฟีเจอร์ของข้อความปัจจุบัน
        temp_df = pd.DataFrame([item])
        features_df = create_features(temp_df)

        # ดึงค่าฟีเจอร์ออกมา
        features_series = features_df.drop(columns=['message', 'label', 'processed_message'], errors='ignore').iloc[0]

        # เรียกใช้ฟังก์ชันหลัก โดยส่งพารามิเตอร์ทั้งหมดที่จำเป็นเข้าไปด้วย
        pred_label, reason = analyze_and_log_sms(sms_text, pipeline_path, encoder_path, LOG_FILE)

        # เก็บผลลัพธ์นำไปสร้าง Report
        true_labels.append(true_label)
        predicted_labels.append(pred_label)

        # แสดงผลการทำนายของแต่ละข้อความพร้อมเหตุผล
        print(f"\nข้อความ  : '{sms_text}'")
        print(f" -> ผลทำนาย : {pred_label} (ค่าจริง: {true_label})")

        # แสดงฟีเจอร์ที่สร้างขึ้น
        print(" -> ฟีเจอร์ที่สร้าง:")
        for col, val in features_series.items():
            print(f"    - {col}: {val}")

        print(f" -> เหตุผลหลัก : {reason}")

    # แสดงรายงาน
    print("\n\nสรุปผลการทดสอบ Classification Report")
    le = joblib.load(encoder_path)
    print(classification_report(true_labels, predicted_labels, target_names=le.classes_))

else:
    print("ไม่เจอ 'test_dataset'")


เริ่มการทดสอบระบบแบบกลุ่ม


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)



ข้อความ  : 'พัสดุของคุณ TH123456789 กำลังจะจัดส่งถึงคุณภายในวันนี้'
 -> ผลทำนาย : OK (ค่าจริง: OK)
 -> ฟีเจอร์ที่สร้าง:
    - message_length: 54.0
    - digit_count: 0.0
    - uppercase_ratio: 0.037037037037037035
    - repeated_punctuation_count: 0.0
    - url_count: 0.0
    - has_short_url: 0.0
    - has_phone: 0.0
    - has_line_keyword: 0.0
    - advertisement_words_count: 0.0
    - is_known_brand: 0.0
    - is_gambling_keyword: 0.0
    - is_loan_keyword: 0.0
    - is_job_offer_keyword: 0.0
    - emotional_words_count: 1.0
    - obfuscated_word_count: 0.0
    - word_diversity_ratio: 0.9999997500000625
 -> เหตุผลหลัก : 'พัสดุ : 0.3429376830559107', '123456789 : 0.31848066400932834', 'จัดส่ง : 0.28612390394743376', '  : 0.22230546413451335', 'กำลังจะ : 0.20264823161138626'


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)



ข้อความ  : 'คำสั่งซื้อ #98765 ของคุณจัดส่งสำเร็จแล้ว ขอบคุณที่ใช้บริการ'
 -> ผลทำนาย : OK (ค่าจริง: OK)
 -> ฟีเจอร์ที่สร้าง:
    - message_length: 59.0
    - digit_count: 0.0
    - uppercase_ratio: 0.0
    - repeated_punctuation_count: 0.0
    - url_count: 0.0
    - has_short_url: 0.0
    - has_phone: 0.0
    - has_line_keyword: 0.0
    - advertisement_words_count: 0.0
    - is_known_brand: 0.0
    - is_gambling_keyword: 0.0
    - is_loan_keyword: 0.0
    - is_job_offer_keyword: 0.0
    - emotional_words_count: 0.0
    - obfuscated_word_count: 0.0
    - word_diversity_ratio: 0.9999998333333611
 -> เหตุผลหลัก : 'สำเร็จ : 0.38282641642286447', 'ขอบคุณ : 0.2818321707859275', 'บริการ : 0.23969358065707888', 'สั่งซื้อ : 0.21501522947431095', 'จัดส่ง : 0.2120244636474275'


...


ข้อความ  : 'โปรสุดคุ้ม! เน็ตไม่อั้น โทรฟรีเพียงวันละ 9 บาท สมัครกด *123#'
 -> ผลทำนาย : spam (ค่าจริง: spam)
 -> ฟีเจอร์ที่สร้าง:
    - message_length: 60.0
    - digit_count: 0.0
    - uppercase_ratio: 0.0
    - repeated_punctuation_count: 0.0
    - url_count: 0.0
    - has_short_url: 0.0
    - has_phone: 0.0
    - has_line_keyword: 0.0
    - advertisement_words_count: 0.0
    - is_known_brand: 0.0
    - is_gambling_keyword: 0.0
    - is_loan_keyword: 0.0
    - is_job_offer_keyword: 0.0
    - emotional_words_count: 1.0
    - obfuscated_word_count: 0.0
    - word_diversity_ratio: 0.9999999090909174
 -> เหตุผลหลัก : 'โทร : 0.2893151551098348', 'คุ้ม : 0.25942368490055956', 'เน็ต : 0.23706560151951472', 'ฟรี : 0.23436749848861674', 'กด : 0.21542580336580064'


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)



ข้อความ  : 'โหลดแอปดูดวงฟรี แม่นขั้นเทพ พร้อมคำทำนายเฉพาะคุณ >> www.horolive.today'
 -> ผลทำนาย : spam (ค่าจริง: spam)
 -> ฟีเจอร์ที่สร้าง:
    - message_length: 70.0
    - digit_count: 0.0
    - uppercase_ratio: 0.0
    - repeated_punctuation_count: 0.0
    - url_count: 0.0
    - has_short_url: 0.0
    - has_phone: 0.0
    - has_line_keyword: 0.0
    - advertisement_words_count: 0.0
    - is_known_brand: 0.0
    - is_gambling_keyword: 0.0
    - is_loan_keyword: 0.0
    - is_job_offer_keyword: 0.0
    - emotional_words_count: 1.0
    - obfuscated_word_count: 0.0
    - word_diversity_ratio: 0.9999999090909174
 -> เหตุผลหลัก : 'โหลด : 0.33439048998138166', 'แม่น : 0.29495625350530874', '.horolive.today : 0.24967653323029249', 'ดูดวง : 0.22900481320496288', 'ฟรี : 0.20901132798247418'


/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/tmp/ipython-input-3929519293.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)



ข้อความ  : 'ดูหนังฟรีไม่อั้นเดือนแรก สมัครเลยวันนี้'
 -> ผลทำนาย : spam (ค่าจริง: spam)
 -> ฟีเจอร์ที่สร้าง:
    - message_length: 39.0
    - digit_count: 0.0
    - uppercase_ratio: 0.0
    - repeated_punctuation_count: 0.0
    - url_count: 0.0
    - has_short_url: 0.0
    - has_phone: 0.0
    - has_line_keyword: 0.0
    - advertisement_words_count: 0.0
    - is_known_brand: 0.0
    - is_gambling_keyword: 0.0
    - is_loan_keyword: 0.0
    - is_job_offer_keyword: 0.0
    - emotional_words_count: 1.0
    - obfuscated_word_count: 0.0
    - word_diversity_ratio: 0.9999998333333611
 -> เหตุผลหลัก : 'หนัง : 0.31515818776503407', 'เดือน : 0.3066501172552131', 'ฟรี : 0.2549078630398719', 'ดู : 0.22343590910884353', 'วันนี้ : 0.1911320217622177'


สรุปผลการทดสอบ Classification Report
              precision    recall  f1-score   support

          OK       1.00      1.00      1.00         5
         OTP       1.00      0.80      0.89         5
        scam       0.83      1.00      0.91      

# HTML ทดสอบระบบ

In [ ]:
!pip install flask pandas pyngrok pythainlp lime scikit-learn imbalanced-learn -q

In [ ]:
!ngrok config add-authtoken ..................

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
from google.colab import drive

print("กำลังเชื่อมต่อ Drive")
drive.mount('/content/drive', force_remount=True)

DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SMS_Scam_Final_Project/'
TEMPLATE_DIR = os.path.join(DRIVE_PROJECT_PATH, 'templates')
html_file_path = os.path.join(TEMPLATE_DIR, 'index.html')

os.makedirs(TEMPLATE_DIR, exist_ok=True)
print(f"สำเร็จที่ '{TEMPLATE_DIR}' ")

กำลังเชื่อมต่อ Drive
Mounted at /content/drive
สำเร็จที่ '/content/drive/MyDrive/SMS_Scam_Final_Project/templates' 


In [ ]:
%%writefile /content/drive/MyDrive/SMS_Scam_Final_Project/templates/index.html
<!DOCTYPE html>
<html lang="th">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>SMS Analyzer - วิเคราะห์ข้อความ</title>
    <link href="https://fonts.googleapis.com/css2?family=Kanit:wght@300;400;500;700&display=swap" rel="stylesheet">
    <link href="https://unpkg.com/aos@2.3.1/dist/aos.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        :root {
            --bg-dark: #121418;
            --bg-medium: #1a1c22;
            --bg-light: #2c2f3a;
            --text-primary: #f0f0f0;
            --text-secondary: #a0a0a0;
            --accent-primary: #4f46e5;
            --accent-secondary: #7c3aed;
            --border-color: rgba(255, 255, 255, 0.1);
            --success-color: #22c55e;
            --warning-color: #f59e0b;
            --danger-color: #ef4444;
            --info-color: #3b82f6;
        }

        * { box-sizing: border-box; }

        body {
            font-family: 'Kanit', sans-serif;
            background-color: var(--bg-dark);
            color: var(--text-primary);
            margin: 0;
            padding: 2rem 1rem;
            display: flex;
            flex-direction: column;
            align-items: center;
            min-height: 100vh;
            background-image: radial-gradient(circle at 1px 1px, rgba(255,255,255,0.05) 1px, transparent 0);
            background-size: 20px 20px;
        }

        .container {
            width: 100%;
            max-width: 800px;
            background: rgba(26, 28, 34, 0.8);
            backdrop-filter: blur(12px);
            padding: 2rem;
            border-radius: 24px;
            border: 1px solid var(--border-color);
            box-shadow: 0 10px 40px rgba(0, 0, 0, 0.5);
        }

        .header {
            text-align: center;
            margin-bottom: 2.5rem;
        }
        .header h1 {
            font-size: 2.8rem;
            font-weight: 700;
            margin: 0;
            background: -webkit-linear-gradient(45deg, var(--accent-secondary), var(--accent-primary));
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }
        .header .tagline {
            font-size: 1.1rem;
            color: var(--text-secondary);
            margin-top: 0.5rem;
        }

        textarea {
            font-family: 'Kanit', sans-serif;
            font-size: 1rem;
            width: 100%;
            min-height: 150px;
            padding: 1rem;
            border-radius: 12px;
            border: 1px solid var(--bg-light);
            background-color: var(--bg-medium);
            color: var(--text-primary);
            resize: vertical;
            transition: all 0.3s ease;
        }
        textarea:focus {
            outline: none;
            border-color: var(--accent-primary);
            box-shadow: 0 0 0 3px rgba(79, 70, 229, 0.5);
        }

        button {
            font-family: 'Kanit', sans-serif;
            font-size: 1.2rem;
            font-weight: 500;
            width: 100%;
            padding: 1rem;
            border: none;
            border-radius: 12px;
            background: linear-gradient(90deg, var(--accent-primary), var(--accent-secondary));
            color: #ffffff;
            cursor: pointer;
            transition: all 0.3s ease;
            margin-top: 1rem;
            display: flex;
            align-items: center;
            justify-content: center;
            gap: 0.5rem;
        }
        button:hover {
            transform: translateY(-3px);
            box-shadow: 0 8px 20px rgba(124, 58, 237, 0.3);
        }
        button:disabled {
            cursor: not-allowed;
            background: var(--bg-light);
        }

        #loader {
            display: none;
            margin-top: 1.5rem;
            text-align: center;
        }
        .spinner {
            border: 4px solid var(--bg-light);
            border-top: 4px solid var(--accent-primary);
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto 1rem;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        #results-container {
            display: none;
            margin-top: 2.5rem;
            border-top: 1px solid var(--border-color);
            padding-top: 2rem;
        }

        .result-grid {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 1.5rem;
            align-items: start;
        }

        .result-card {
            background: var(--bg-medium);
            padding: 1.5rem;
            border-radius: 16px;
            border: 1px solid var(--border-color);
        }
        .result-card h3 {
            margin-top: 0;
            margin-bottom: 1rem;
            font-size: 1.2rem;
            border-bottom: 1px solid var(--border-color);
            padding-bottom: 0.75rem;
        }
        #main-result h2 {
            font-size: 2rem;
            margin: 0;
            font-weight: 700;
            text-align: center;
        }
        #main-result p {
            text-align: center;
            color: var(--text-secondary);
            margin-top: 0.25rem;
        }
        #main-result .icon {
            font-size: 3rem;
            text-align: center;
            margin-bottom: 1rem;
        }
        .OK { color: var(--success-color); }
        .OTP { color: var(--info-color); }
        .spam { color: var(--warning-color); }
        .scam { color: var(--danger-color); }

        #reason-list {
            list-style-type: none;
            padding: 0;
            margin: 0;
        }
        #reason-list li {
            background-color: var(--bg-light);
            padding: 0.5rem 1rem;
            border-radius: 8px;
            margin-bottom: 0.5rem;
            font-family: 'Menlo', 'Consolas', monospace;
            font-size: 0.9rem;
            display: flex;
            justify-content: space-between;
        }
        #reason-list .weight {
            color: var(--text-secondary);
        }

        @media (max-width: 768px) {
            .result-grid {
                grid-template-columns: 1fr;
            }
            .header h1 { font-size: 2.2rem; }
            body { padding: 1rem 0.5rem; }
            .container { padding: 1.5rem; }
        }

    </style>
</head>
<body>
    <div class="container" data-aos="fade-up">
        <div class="header">
            <h1>SMS Analyzer 🛡️</h1>
            <p class="tagline">ระบบ AI ตรวจจับข้อความ Spam และ Scam</p>
        </div>

        <form id="sms-form">
            <textarea id="sms-input" name="sms_message" placeholder="วางข้อความ SMS ของคุณที่นี่..." required></textarea>
            <button type="submit" id="submit-btn">
                <svg xmlns="http://www.w3.org/2000/svg" width="24" height="24" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="m21.73 18-8-14a2 2 0 0 0-3.46 0l-8 14A2 2 0 0 0 4 21h16a2 2 0 0 0 1.73-3Z"></path><path d="M12 9v4"></path><path d="M12 17h.01"></path></svg>
                <span>วิเคราะห์ข้อความ</span>
            </button>
        </form>

        <div id="loader">
            <div class="spinner"></div>
            <p>กำลังวิเคราะห์...</p>
        </div>

        <div id="results-container" data-aos="fade-up" data-aos-delay="200">
            <div class="result-grid">
                <div class="result-card" id="main-result">
                    </div>
                <div class="result-card">
                    <h3>📊 ระดับความน่าจะเป็น</h3>
                    <canvas id="probaChart"></canvas>
                </div>
                <div class="result-card">
                    <h3>🔑 เหตุผลหลักในการตัดสินใจ</h3>
                    <ul id="reason-list">
                        </ul>
                </div>
                <div class="result-card">
                    <h3>⚙️ ฟีเจอร์ที่สกัดได้</h3>
                    <pre id="features-pre"></pre>
                </div>
            </div>
        </div>
    </div>

    <script src="https://unpkg.com/aos@2.3.1/dist/aos.js"></script>
    <script>
        AOS.init({ duration: 800, once: true });

        const form = document.getElementById('sms-form');
        const smsInput = document.getElementById('sms-input');
        const submitBtn = document.getElementById('submit-btn');
        const loader = document.getElementById('loader');
        const resultsContainer = document.getElementById('results-container');

        const mainResultEl = document.getElementById('main-result');
        const reasonListEl = document.getElementById('reason-list');
        const featuresPreEl = document.getElementById('features-pre');
        const probaChartCanvas = document.getElementById('probaChart');
        let probaChart = null;

        form.addEventListener('submit', async (e) => {
            e.preventDefault();
            resultsContainer.style.display = 'none';
            loader.style.display = 'block';
            submitBtn.disabled = true;
            submitBtn.querySelector('span').textContent = "กำลังประมวลผล...";

            const formData = new FormData(form);

            try {
                const response = await fetch('/predict', { method: 'POST', body: formData });
                const data = await response.json();

                if (data.error) {
                    alert('เกิดข้อผิดพลาด: ' + data.error);
                } else {
                    displayResults(data);
                }
            } catch (error) {
                alert('เกิดข้อผิดพลาดในการเชื่อมต่อกับเซิร์ฟเวอร์');
            } finally {
                loader.style.display = 'none';
                submitBtn.disabled = false;
                submitBtn.querySelector('span').textContent = "วิเคราะห์ข้อความ";
            }
        });

        function displayResults(data) {
            const icons = { 'OK': '✅', 'OTP': '🔑', 'spam': '📢', 'scam': '💀' };
            const messages = {
                'OK': 'ข้อความนี้เป็นข้อความทั่วไป',
                'OTP': 'ข้อความนี้เป็นรหัสผ่านใช้ครั้งเดียว',
                'spam': 'ข้อความนี้มีลักษณะเป็นสแปม/โฆษณา',
                'scam': 'ข้อความนี้มีความเสี่ยงเป็นมิจฉาชีพ!'
            };
            mainResultEl.innerHTML = `
                <div class="icon">${icons[data.prediction] || '❓'}</div>
                <h2 class="${data.prediction}">${data.prediction}</h2>
                <p>${messages[data.prediction] || 'ไม่สามารถระบุประเภทได้'}</p>
            `;

            if (probaChart) { probaChart.destroy(); }
            const chartData = {
                labels: data.probabilities.map(p => p.label),
                datasets: [{
                    label: 'ความน่าจะเป็น (%)',
                    data: data.probabilities.map(p => p.score),
                    backgroundColor: [
                        'rgba(34, 197, 94, 0.6)',
                        'rgba(59, 130, 246, 0.6)',
                        'rgba(245, 158, 11, 0.6)',
                        'rgba(239, 68, 68, 0.6)',
                    ],
                    borderColor: [
                        '#22c55e', '#3b82f6', '#f59e0b', '#ef4444'
                    ],
                    borderWidth: 1
                }]
            };
            probaChart = new Chart(probaChartCanvas, {
                type: 'bar',
                data: chartData,
                options: {
                    indexAxis: 'y',
                    scales: {
                        x: { beginAtZero: true, max: 100 }
                    },
                    plugins: { legend: { display: false } }
                }
            });

            let reasonHtml = '';
            const reasons = data.reason.split(', ').map(r => r.replace(/'/g, ''));
            if(data.reason && data.reason !== "ไม่พบคำสำคัญ") {
                reasons.forEach(r => {
                    const [word, weight] = r.split(' : ');
                    reasonHtml += `<li><span>${word}</span><span class="weight">${parseFloat(weight).toFixed(2)}</span></li>`;
                });
            } else {
                 reasonHtml = `<li>ไม่พบคำสำคัญที่ชัดเจน</li>`;
            }
            reasonListEl.innerHTML = reasonHtml;

            let featuresText = '';
            for (const [key, value] of Object.entries(data.features)) {
                const formattedValue = typeof value === 'number' ? value.toFixed(4) : value;
                featuresText += `${key.padEnd(28)}: ${formattedValue}\n`;
            }
            featuresPreEl.textContent = featuresText;

            resultsContainer.style.display = 'block';
            resultsContainer.scrollIntoView({ behavior: 'smooth', block: 'start' });
            AOS.refresh();
        }
    </script>
</body>
</html>

Writing /content/drive/MyDrive/SMS_Scam_Final_Project/templates/index.html


In [ ]:
%%writefile /content/drive/MyDrive/SMS_Scam_Final_Project/app.py
import os, csv, joblib, re, string
from flask import Flask, request, jsonify, render_template
import pandas as pd
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
import lime
import lime.lime_text

# Setup Paths and Flask App
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/SMS_Scam_Final_Project/'
TEMPLATE_DIR = os.path.join(DRIVE_PROJECT_PATH, 'templates')
app = Flask(__name__, template_folder=TEMPLATE_DIR)

# เปลี่ยนไปใช้โมเดลเวอร์ชันล่าสุดที่รวมทุกอย่างแล้ว
pipeline_path = os.path.join(DRIVE_PROJECT_PATH, 'final_sms_classifier_production.joblib')
encoder_path = os.path.join(DRIVE_PROJECT_PATH, 'label_encoder.joblib')


# Load Models
try:
    pipeline = joblib.load(pipeline_path)
    le = joblib.load(encoder_path)
    print("โหลดโมเดลและตัวเข้ารหัสแล้ว")
except Exception as e:
    print(f"Error : {e}")
    pipeline, le = None, None

# Helper Function for Text Processing
def text_process(text: str) -> str:
    if not isinstance(text, str):
        return ""
    emoji_pattern = re.compile(
        "["
        u"\\U0001F600-\\U0001F64F" u"\\U0001F300-\\U0001F5FF" u"\\U0001F680-\\U0001F6FF"
        u"\\U0001F1E0-\\U0001F1FF" u"\\U00002702-\\U000027B0" u"\\U000024C2-\\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    tokens = word_tokenize(text, engine='newmm')
    stopwords_list = list(thai_stopwords())
    punctuations_to_remove = string.punctuation.replace('%', '')
    processed_tokens = [word for word in tokens if word not in stopwords_list and word not in punctuations_to_remove]
    return " ".join(processed_tokens)

# Feature Creation Function
def create_features(df_input: pd.DataFrame) -> pd.DataFrame:
    df_out = df_input.copy()
    df_out['message'] = df_out['message'].astype(str)
    df_out['message_length'] = df_out['message'].str.len()
    df_out['digit_count'] = df_out['message'].str.count(r'\\d')
    df_out['uppercase_ratio'] = df_out['message'].str.count(r'[A-Z]') / (df_out['message_length'] + 1e-6)
    df_out['repeated_punctuation_count'] = df_out['message'].str.count(r'([!?.])\\1{1,}')
    df_out['url_count'] = df_out['message'].str.count(r'https?://\\S+|www\\.\\S+|\\S+\\.\\S+')
    short_url_pattern = 'bit.ly|cutt.ly|t.co|rebrand.ly|shorturl.asia|line.me|lin.ee'
    df_out['has_short_url'] = df_out['message'].str.contains(short_url_pattern, case=False, regex=True).astype(int)
    phone_pattern = r'(\\d{2,3}-\\d{3,4}-\\d{4}|\\d{9,10})'
    df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
    df_out['has_line_keyword'] = df_out['message'].str.contains('Line|ไลน์|แอดไลน์', case=False, regex=True).astype(int)
    advertisement_keywords = ['โปรโมชั่น', 'ส่วนลด', 'ช้อป', 'สาขา', 'คลิกเลย', 'แบรนด์', 'คอลเลคชั่น', 'โค้ด']
    df_out['advertisement_words_count'] = df_out['message'].str.count('|'.join(advertisement_keywords)).astype(int)
    brand_keywords = ['Lazada', 'Shopee', 'AIS', 'dtac', 'True', 'KFC', 'Grab', 'Central', 'Uniqlo', 'SCB', 'KBank']
    df_out['is_known_brand'] = df_out['message'].str.contains('|'.join(brand_keywords), case=False, regex=True).astype(int)
    df_out['is_gambling_keyword'] = df_out['message'].str.contains('เว็บตรง|PG|JOKER|สล็อต|บาคาร่า|คาสิโน|แทงหวย|ฝากถอน|ยูสใหม่|แตกง่าย', case=False, regex=True).astype(int)
    df_out['is_loan_keyword'] = df_out['message'].str.contains('สินเชื่อ|เงินด่วน|อนุมัติไว|วงเงินกู้|กู้ได้|ไม่เช็คบูโร', case=False, regex=True).astype(int)
    df_out['is_job_offer_keyword'] = df_out['message'].str.contains('ทำงานที่บ้าน|รายได้ต่อวัน|part-time|พาร์ทไทม์|รับสมัคร', case=False, regex=True).astype(int)
    emotional_words = ['ฟรี', 'รางวัล', 'แจก', 'โบนัส', 'เครดิต', 'รับสิทธิ์', 'เงินคืน', 'ด่วน', 'ภายใน', 'จำกัด', 'เท่านั้น', 'ผิดปกติ', 'ถูกระงับ', 'มีปัญหา']
    df_out['emotional_words_count'] = df_out['message'].str.count('|'.join(emotional_words)).astype(int)
    obfuscated_pattern = r'\\b[ก-ฮ]*[a-zA-Z0-9]+[ก-ฮ]+[a-zA-Z0-9]*\\b'
    df_out['obfuscated_word_count'] = df_out['message'].str.count(obfuscated_pattern)
    df_out['processed_message'] = df_out['message'].apply(text_process)
    df_out['word_diversity_ratio'] = df_out['processed_message'].apply(
        lambda x: len(set(x.split())) / (len(x.split()) + 1e-6)
    )
    return df_out

# Main Prediction Function
def analyze_sms(sms_text):
    if not pipeline or not le:
        return {"error": "Model not loaded."}
    try:
        features_df = create_features(pd.DataFrame([{'message': sms_text}]))
        prediction_encoded = pipeline.predict(features_df)[0]
        prediction = le.inverse_transform([prediction_encoded])[0]
        probabilities = pipeline.predict_proba(features_df)[0]
        prob_list = [{"label": class_name, "score": round(probabilities[i] * 100, 2)} for i, class_name in enumerate(le.classes_)]
        prob_list.sort(key=lambda x: x['score'], reverse=True)

        def lime_predict_fn(texts):
            lime_df = create_features(pd.DataFrame(texts, columns=['message']))
            return pipeline.predict_proba(lime_df)

        explainer = lime.lime_text.LimeTextExplainer(class_names=le.classes_, bow=False, split_expression=word_tokenize)
        exp = explainer.explain_instance(sms_text, lime_predict_fn, num_features=5, labels=[prediction_encoded])

        reason_parts = [f"'{word} : {weight:.4f}'" for word, weight in exp.as_list(label=prediction_encoded) if weight > 0]
        reason_string = ", ".join(reason_parts) if reason_parts else "ไม่พบคำสำคัญ"

        features_dict = features_df.drop(columns=['message', 'processed_message'], errors='ignore').iloc[0].to_dict()

        return {
            "prediction": prediction,
            "probabilities": prob_list,
            "reason": reason_string,
            "features": features_dict
        }
    except Exception as e:
        return {"error": str(e)}

# Routes
@app.route('/')
def home():
    if not os.path.exists(TEMPLATE_DIR):
        os.makedirs(TEMPLATE_DIR)
    html_file_path = os.path.join(TEMPLATE_DIR, 'index.html')
    if not os.path.exists(html_file_path):
        with open(html_file_path, 'w') as f:
            f.write('<!DOCTYPE html><html><head><title>Analyzer</title></head><body><h1>Update index.html!</h1></body></html>')

    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'sms_message' not in request.form:
        return jsonify({"error": "Missing 'sms_message' in form data."}), 400
    sms_text = request.form['sms_message']
    if not sms_text.strip():
        return jsonify({"error": "SMS message cannot be empty."}), 400

    result = analyze_sms(sms_text)
    return jsonify(result)

Writing /content/drive/MyDrive/SMS_Scam_Final_Project/app.py


In [ ]:
from pyngrok import ngrok
import sys

PROJECT_DIR = "/content/drive/MyDrive/SMS_Scam_Final_Project"
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

from app import app

public_url = ngrok.connect(5000)
print("---" * 10)
print(f"Click here! : {public_url}")
print("---" * 10)

app.run(port=5000)

โหลดโมเดลและตัวเข้ารหัสแล้ว
------------------------------
Click here! : NgrokTunnel: "https://cabd0658b2ad.ngrok-free.app" -> "http://localhost:5000"
------------------------------
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 07:54:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 07:54:56] "GET /favicon.ico HTTP/1.1" 404 -
/content/drive/MyDrive/SMS_Scam_Final_Project/app.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
/content/drive/MyDrive/SMS_Scam_Final_Project/app.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_out['has_phone'] = df_out['message'].str.contains(phone_pattern, regex=True).astype(int)
INFO:werkzeug:127.0.0.1 - - [07/Aug/2025 07:55:08] "POST /predict HTT